In [17]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(URL)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [174]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value = 'ALL',
                                             placeholder = 'Select Launch Site',
                                             searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 1000: '1000', 2000: '2000',
                                                      3000: '3000', 4000: '4000', 5000: '5000',
                                                      6000: '6000', 7000: '7000',
                                                      8000: '8000', 9000: '9000', 10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig1 = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig1
    else:
        entered_site_df = spacex_df[spacex_df['Launch Site'] == entered_site]['class'].value_counts().reset_index()
        entered_site_df = entered_site_df.rename(columns= {'index': 'Class', 'class': 'Counts'})
        fig1 = px.pie(entered_site_df, values='Counts', 
        names='Class', 
        title= f'Total Sucess Launches for Site {entered_site}')
        return fig1

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_plot(entered_site, payload_value):
    if entered_site == 'ALL':
        fig2 = px.scatter(spacex_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category',
                         title = 'Colleration between Payload and Success for all Sites')
        return fig2
    else:
        min_value, max_value = payload_value
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site][['class', 'Payload Mass (kg)', 'Booster Version Category']]
        filter_df = site_df.loc[(site_df['Payload Mass (kg)']>= min_value)&(site_df['Payload Mass (kg)']<= max_value)]
        fig2 = px.scatter(filter_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category',
                         title = f'Colleration between Payload and Success for site {entered_site} and Payload Mass of Range {payload_value}')
        return fig2
        


# Run the app
if __name__ == '__main__':
    app.run_server()
